## 325 鄭書承
# 目標：
1. 利用爬蟲抓取台股歷史資料
2. 配合交易方式打造一個回測並可以持續更新的程式
3. 在每日收盤結束後自動更新資料
4. 若有符合買進或賣出的點會透過LINE通知明天應該買入或賣出多少股票

In [1]:
def lineNotify(stock_id, flag, shares, line_token):
    import requests
    msg = f"{flag}{stock_id} {shares}股"
    stickerPackageId = 2
    stickerId = 34
    url = "https://notify-api.line.me/api/notify"
    headers = {
        "Authorization": "Bearer " + line_token
    }
   
    payload = {"message": msg, "stickerPackageId": stickerPackageId, 'stickerId': stickerId}
    r = requests.post(url, headers = headers, params = payload)
    return r.status_code

In [2]:
def goodinfo_crawler(stock_id):
    import requests
    from bs4 import BeautifulSoup
    headers = {
        "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Mobile Safari/537.36"
    }
    req = requests.get(f"https://goodinfo.tw/StockInfo/ShowBuySaleChart.asp?STOCK_ID={stock_id}&CHT_CAT=DATE", headers = headers)
    req.encoding = 'utf-8'
    soup = BeautifulSoup(req.text, "html.parser")
    date = soup.select("#row0 > td:nth-child(1)")[0].text
    return {
        "foreign_investor":soup.select("#row0 > td:nth-child(8)")[0].text,   #當天外資買賣張數
        "investment_trust":soup.select("#row0 > td:nth-child(13)")[0].text   #當天投信買賣張數
    }, date


In [17]:
def goodinfo_crawler1(stock_id, df):
    import requests
    from bs4 import BeautifulSoup
    from datetime import datetime
    import pandas as pd
    headers = {
        "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Mobile Safari/537.36"
    }
    url1 = f"https://goodinfo.tw/StockInfo/ShowBuySaleChart.asp?STOCK_ID={stock_id}&CHT_CAT=DATE"
    url2 = f"https://goodinfo.tw/StockInfo/ShowK_Chart.asp?STOCK_ID={stock_id}&CHT_CAT2=DATE"
    req1 = requests.get(url1, headers = headers)
    req1.encoding = 'utf-8'
    soup1 = BeautifulSoup(req1.text, "html.parser")
    req2 = requests.get(url2, headers = headers)
    req2.encoding = 'utf-8'
    soup2 = BeautifulSoup(req2.text, "html.parser")
    data_last_date = datetime.date(datetime.strptime(df.index[-1], "%Y-%m-%d"))
    count = 0
    while True:
        date = datetime.strptime(soup1.select(f"#row{count} > td:nth-child(1)")[0].text, "%y'%m/%d")
        date = datetime.date(date)
        if date == data_last_date:
            break
        else:
            count += 1
    for i in range(count-1, -1, -1):
        data = { 
            "priceOpen":soup2.select(f"#row{i} > td:nth-child(2) > nobr")[0].text,
            "priceHigh":soup2.select(f"#row{i} > td:nth-child(3) > nobr")[0].text,
            "priceLow":soup2.select(f"#row{i} > td:nth-child(4) > nobr")[0].text,
            "priceClose":soup2.select(f"#row{i} > td:nth-child(5) > nobr")[0].text,
            "foreign_investor":soup1.select(f"#row{i} > td:nth-child(8)")[0].text.lstrip("+").replace(",", ""),
            "investment_trust":soup1.select(f"#row{i} > td:nth-child(13)")[0].text.lstrip("+").replace(",", ""), 
        }
        date = soup1.select(f"#row{i} > td:nth-child(1)")[0].text
        date = datetime.date(datetime.strptime(date, "%y'%m/%d"))
        ser = pd.Series(data, name = date)
        df = df.append(ser)
        df = df.astype('float64')
    return df


In [5]:
import datetime
datetime.datetime.date(datetime.datetime.strptime("20'04/06", '%y\'%m/%d'))

datetime.date(2020, 4, 6)

In [88]:
for i in range(10, -1, -1):
    print(i)

10
9
8
7
6
5
4
3
2
1
0


In [4]:
def fugle_api_crawler(stock_id, fugle_token): 
    import requests
    payload = {
        "symbolId":f"{stock_id}",
        "apiToken":fugle_token
    }
    req1 = requests.get("https://api.fugle.tw/realtime/v0/intraday/quote?", params = payload)
    json_data1 = req1.json()['data']['quote']
    req2 = requests.get("https://api.fugle.tw/realtime/v0/intraday/meta?", params = payload)
    json_data2 = req2.json()['data']['meta']
    #當天的開高低收
    return {
        'priceOpen':json_data1['priceOpen']['price'],
        'priceHigh':json_data1['priceHigh']['price'],
        'priceLow':json_data1['priceLow']['price'],
        'priceClose':json_data1['trade']['price'],
    }

In [60]:
def goodinfo_crawler_pastdata(stock_id):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    from selenium import webdriver
    from selenium.webdriver.support.ui import Select
    import time
    from datetime import datetime
    url1 = f"https://goodinfo.tw/StockInfo/ShowBuySaleChart.asp?STOCK_ID={stock_id}&CHT_CAT=DATE"
    url2 = f"https://goodinfo.tw/StockInfo/ShowK_Chart.asp?STOCK_ID={stock_id}&CHT_CAT2=DATE"
    driver1 = webdriver.Chrome()
    driver2 = webdriver.Chrome()
    driver1.implicitly_wait(5)
    driver2.implicitly_wait(5)
    driver1.get(url1)
    driver2.get(url2)
    #抓取歷史法人買賣張數
    s1 = Select(driver1.find_element_by_css_selector('#divBuySaleDetailData > table > tbody > tr > td > table > tbody > tr > td:nth-child(2) > nobr:nth-child(1) > select'))
    #抓取歷史股價
    s2 = Select(driver2.find_element_by_css_selector('#selK_ChartPeriod'))
    s1.select_by_visible_text('一年')
    s2.select_by_visible_text('一年')
    time.sleep(5)
    soup1 = BeautifulSoup(driver1.page_source, 'html.parser')
    soup2 = BeautifulSoup(driver2.page_source, 'html.parser')
    df = pd.DataFrame({})
    i = 0
    while True:
        try:
            data = { 
                "priceOpen":soup2.select(f"#row{i} > td:nth-child(2) > nobr")[0].text,
                "priceHigh":soup2.select(f"#row{i} > td:nth-child(3) > nobr")[0].text,
                "priceLow":soup2.select(f"#row{i} > td:nth-child(4) > nobr")[0].text,
                "priceClose":soup2.select(f"#row{i} > td:nth-child(5) > nobr")[0].text,
                "foreign_investor":soup1.select(f"#row{i} > td:nth-child(8)")[0].text.lstrip("+").replace(",", ""),
                "investment_trust":soup1.select(f"#row{i} > td:nth-child(13)")[0].text.lstrip("+").replace(",", ""), 
            }
            date = soup1.select(f"#row{i} > td:nth-child(1)")[0].text
            date = datetime.strptime(date, "%y'%m/%d")
            ser = pd.Series(data, name = date)
            df = df.append(ser)
            print(stock_id, date)
            i += 1
        except:
            break
    for index, row in df.iterrows():
        if (df.loc[index, "foreign_investor"] == "") or (df.loc[index, "investment_trust"] == ""):
            df = df.drop(index)
    driver1.close()
    driver2.close()
    df = df.sort_index()
    df = df.astype('float64')
    return df


In [123]:
def trading1(df, balance, stock_id, fee, tax, line_token):
    import pandas as pd
    import numpy as np
    df['buy_or_sell'] = np.zeros(len(df)).reshape((len(df)), 1)    #先將明天買賣股數設為0
    df['shares'] = np.zeros(len(df)).reshape((len(df)), 1)   #先將股數設為0
    df['balance'] = np.zeros(len(df)).reshape((len(df)), 1)  #先將餘額設為0
    df.iloc[0, -1] = balance   #將一開始的餘額設成輸入的值
    df['income'] = np.zeros(len(df)).reshape((len(df)), 1)   #先將損益設為0
    df['profit'] = np.zeros(len(df)).reshape((len(df)), 1)   #先將報酬率設為0
    df['last_close'] = df['priceClose'].shift(1)  #今天收盤價是明天的昨收價
    df['ten_ma'] = df['priceClose'].rolling(10).mean()  #十日平均
    df['IT_flag'] = df['investment_trust'] > 0   #投信是否買超
    df['IT_buy_days'] = df['IT_flag'].rolling(3).sum()   #投信這三天買超的天數
    #當投信連續買超 且 外資今天也買超 且 大於十日平均 就設為買進訊號
    df['buy_flag'] = (df['IT_buy_days'] == 3) & (df['foreign_investor'] > 0) & (df['priceClose'] > df['ten_ma'])
    #當投信不再連續買超 或 收盤價小於十日平均 就設為賣出訊號
    df['sell_flag'] = (df['investment_trust'] < 0) | (df['priceClose'] < df['ten_ma'])   
    count = 0
    win_times = 0 
    before_income = []
    for index, row in df.iterrows():
        #因為第一天沒有昨天的資料所以會出現Error，因此使用try
        try:
            df.loc[index, 'shares'] = yesterday.shares    #先將今天的股數設成和昨天一樣若有買賣再運算
            df.loc[index, 'balance'] = yesterday.balance  #先將今天的餘額設成和昨天一樣若有買賣再運算
            #當昨天的應買賣股數不等於0時，代表今天會執行買賣
            #(附註1：因為三大法人買賣超收盤之後才會知道因此所有動作都只能隔一天才能執行)
            if yesterday.buy_or_sell != 0 :
                df.loc[index, 'buy_or_sell'] = 0    #將應買賣股數變回0
                df.loc[index, 'shares'] = yesterday.shares + yesterday.buy_or_sell   #今日庫存股數為昨日股數加上應買股數
                #今日餘額為昨日餘額加上今天交易股數乘上今天開盤價扣掉手續費和證交稅
                df.loc[index, 'balance'] = yesterday.balance - yesterday.buy_or_sell*df.loc[index, 'priceOpen']*(1- (fee + tax) / 100)  
               
            #當今天買進訊號出現的時候 且 今天的餘額夠買一張股票 明天開盤就買進
            #(附註2：應該使用明天的開盤價，不過明天還沒開盤不會知道開盤價，所以用今天收盤的漲停價來推算餘額夠不夠)
            elif (df.loc[index, 'buy_flag']) and ((df.loc[index, 'balance'] - df.loc[index, 'priceClose'] * 1.1 * 1000) > 0):  
                shares = int(df.loc[index, 'balance'] / (df.loc[index, 'priceClose'] * 1.1 * 1000))   #先估算餘額可以買幾股
                df.loc[index, 'buy_or_sell'] = shares * 1000  #將預估購買的股數填入
                lineNotify(stock_id, "buy", df.loc[index, 'buy_or_sell'], line_token)    #line通知什麼股票應該買多少
            #當今天賣出訊號出現的時候 且 庫存股票股數大於0 明天開盤就賣出
            elif (df.loc[index, 'sell_flag']) and (df.loc[index, 'shares'] > 0):
                df.loc[index, 'buy_or_sell'] = int(df.loc[index, 'shares'] * -1)   #將預估賣出的股數填入
                sell_shares = df.loc[index, 'buy_or_sell']
                lineNotify(stock_id, "sell", df.loc[index, 'buy_or_sell'] * -1, line_token)    #line通知什麼股票應該賣多少
    
            df.loc[index, 'income'] = (df.loc[index, 'priceClose'] * df.loc[index, 'shares'] + df.loc[index, 'balance']) - balance
            df.loc[index, 'profit'] = f"{df.loc[index, 'income']/balance *100:.2f}%"
            if df.loc[index, 'buy_or_sell'] > 0:
                before_income.append(df.loc[index, 'income'])
                count += 1    #買賣次數加1
            elif yesterday.buy_or_sell < 0:
                after_income = df.loc[index, 'income']
                before_income = np.array(before_income)
                win_times += len(before_income[before_income < after_income])
                before_income = []
            yesterday = df.loc[index]    #完成資料更新後將其設為下一天的昨天

        except UnboundLocalError:
            yesterday = df.loc[index]
    print(stock_id, win_times, count)
    print(f"{stock_id}：勝率{win_times/count * 100:.2f}% 報酬率{df.iloc[-1, -7]}")
    return df



In [73]:
import numpy as np
x = np.array([1, 2, 3])
x[x > 1]

array([2, 3])

In [7]:
def backtesting():
    import pygsheets
    import pandas as pd
    import numpy as np
    gc = pygsheets.authorize(service_account_file=r"C:\Users\j3192\Desktop\Jupyter Notebook\台大程式課程\ntu_pyxl_lesson10\pyxl-271804-be731475b523.json")
    wb = gc.open_by_url("https://docs.google.com/spreadsheets/d/1SJ9YRMvoChR8F2nb1H8rxjR5vClyINSIjlcrLsSw6Nk/edit#gid=0")
    wks = wb.worksheet_by_title("追蹤清單")
    stock_list = wks.get_col(1, include_tailing_empty=False)
    fee = float(wks.cell("H1").value)*0.6*2
    tax = float(wks.cell("H2").value)
    balance = int(wks.cell("H3").value)
    fugle_token = wks.cell("K1").value
    line_token = wks.cell("K2").value
    for stock_id in stock_list:
        try:
            wks = wb.worksheet_by_title(str(stock_id))
            cols = len(wks.get_col(1, include_tailing_empty = False))
            df = wks.get_as_df(index_column = 1, end = (cols, 7))
            data1, date = goodinfo_crawler(str(stock_id))
            data2 = fugle_api_crawler(str(stock_id), fugle_token)
            data = {**data1, **data2}    #合併dict
            ser = pd.Series(data, name = date)
            df.append(ser) 
        except: 
            wks = wb.add_worksheet(str(stock_id), rows=100, cols=26, src_tuple=None, index=None)
            df = goodinfo_crawler_pastdata(stock_id)
            wks.set_dataframe(result_df.loc[:, "foreign_investor":"profit"], 'A1', copy_index=True)
        result_df = trading1(df, balance, stock_id, fee, tax, line_token)
        wks.set_dataframe(result_df.loc[:, "foreign_investor":"profit"], 'A1', copy_index=True)
        wks.refresh()

In [132]:
def backtesting1():
    import pygsheets
    import pandas as pd
    import numpy as np
    gc = pygsheets.authorize(service_account_file=r"C:\Users\j3192\Desktop\Jupyter Notebook\台大程式課程\ntu_pyxl_lesson10\pyxl-271804-be731475b523.json")
    wb = gc.open_by_url("https://docs.google.com/spreadsheets/d/1SJ9YRMvoChR8F2nb1H8rxjR5vClyINSIjlcrLsSw6Nk/edit#gid=0")
    wks = wb.worksheet_by_title("追蹤清單")
    stock_list = wks.get_col(1, include_tailing_empty=False)
    fee = float(wks.cell("H1").value)*0.6*2
    tax = float(wks.cell("H2").value)
    balance = int(wks.cell("H3").value)
    fugle_token = wks.cell("K1").value
    line_token = wks.cell("K2").value
    for stock_id in stock_list[-1:]:
        try:
            wks = wb.worksheet_by_title(str(stock_id))
            cols = len(wks.get_col(1, include_tailing_empty = False))
            df = wks.get_as_df(index_column = 1, end = (cols, 7))
            data = goodinfo_crawler1(stock_id ,df)
        except: 
            wks = wb.add_worksheet(str(stock_id), rows=100, cols=26, src_tuple=None, index=None)
            data = goodinfo_crawler_pastdata(stock_id)
        result_df = trading1(data, balance, stock_id, fee, tax, line_token)
        wks.set_dataframe(result_df.loc[:, "foreign_investor":"profit"], 'A1', copy_index=True)
        wks.refresh()
    return result_df

In [133]:
df1 = backtesting1()

2379 2020-04-07 00:00:00
2379 2020-04-06 00:00:00
2379 2020-04-01 00:00:00
2379 2020-03-31 00:00:00
2379 2020-03-30 00:00:00
2379 2020-03-27 00:00:00
2379 2020-03-26 00:00:00
2379 2020-03-25 00:00:00
2379 2020-03-24 00:00:00
2379 2020-03-23 00:00:00
2379 2020-03-20 00:00:00
2379 2020-03-19 00:00:00
2379 2020-03-18 00:00:00
2379 2020-03-17 00:00:00
2379 2020-03-16 00:00:00
2379 2020-03-13 00:00:00
2379 2020-03-12 00:00:00
2379 2020-03-11 00:00:00
2379 2020-03-10 00:00:00
2379 2020-03-09 00:00:00
2379 2020-03-06 00:00:00
2379 2020-03-05 00:00:00
2379 2020-03-04 00:00:00
2379 2020-03-03 00:00:00
2379 2020-03-02 00:00:00
2379 2020-02-27 00:00:00
2379 2020-02-26 00:00:00
2379 2020-02-25 00:00:00
2379 2020-02-24 00:00:00
2379 2020-02-21 00:00:00
2379 2020-02-20 00:00:00
2379 2020-02-19 00:00:00
2379 2020-02-18 00:00:00
2379 2020-02-17 00:00:00
2379 2020-02-14 00:00:00
2379 2020-02-13 00:00:00
2379 2020-02-12 00:00:00
2379 2020-02-11 00:00:00
2379 2020-02-10 00:00:00
2379 2020-02-07 00:00:00


In [49]:
df1

,foreign_investor,investment_trust,priceClose,priceHigh,priceLow,priceOpen,buy_or_sell,shares,balance,income,profit,last_close,ten_ma,IT_flag,IT_buy_days,buy_flag,sell_flag
,,,,,,,,,,,,,,,,,
2019-04-02,6883.0,0.0,20.90,20.90,20.65,20.85,0.0,0.0,1.000000e+06,0.0000,0,NaN,NaN,False,NaN,False,False
2019-04-03,-293.0,0.0,21.25,21.55,21.10,21.15,0.0,0.0,1.000000e+06,0.0000,0.00%,20.90,NaN,False,NaN,False,False
2019-04-08,3039.0,0.0,21.70,21.80,21.35,21.40,0.0,0.0,1.000000e+06,0.0000,0.00%,21.25,NaN,False,0.0,False,False
2019-04-09,-3741.0,16.0,21.70,22.00,21.65,21.85,0.0,0.0,1.000000e+06,0.0000,0.00%,21.70,NaN,True,1.0,False,False
2019-04-10,-704.0,0.0,21.90,22.25,21.80,21.85,0.0,0.0,1.000000e+06,0.0000,0.00%,21.70,NaN,False,1.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-27,-2189.0,-3235.0,25.90,28.00,25.85,27.75,0.0,0.0,1.240313e+06,240312.7705,24.03%,26.45,24.890,False,1.0,False,True
2020-03-30,-4392.0,-3722.0,25.60,25.80,24.70,24.85,0.0,0.0,1.240313e+06,240312.7705,24.03%,25.90,24.730,False,0.0,False,True
2020-03-31,-1053.0,-7.0,25.50,26.40,25.20,26.15,0.0,0.0,1.240313e+06,240312.7705,24.03%,25.60,24.745,False,0.0,False,True


In [21]:
import xlwings as xw
xw.view(df1)

![](r"C:\Users\j3192\Desktop\擷取.PNG")

![](擷取.PNG)

![](擷取1.PNG)

# 總結
### 缺點
1. 原先想要將程式架構在GCP上，並用Google Scheduler每天自動更新資料，但因為爬取的網站需要下拉式選單選取資料時間長度，而我只想到用Selenium方式抓取資料，但無法用Cloud Function上開啟一個模擬瀏覽器
2. 利用Selenium的方式爬蟲速度緩慢，且用for迴圈一列一列抓取速度又更慢，不過有歷史資料的網站不是沒有法人買賣超，就是資料被隱藏或會阻擋爬蟲  

### 解決方案
將Cloud Function中使用到Selenium的程式碼移除，並利用Google Scheduler每天自動更新資料，但當追蹤清單想加入新的股票時需先手動執行程式  
(嘗試使用GCP Computer Engine設定排程執行，但有點不太知道該怎麼處理)